<a href="https://colab.research.google.com/github/darkwingpatil/Ml_hackethons/blob/main/Langchain_Router_Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **LangChain 🦜🔗:  Routing**


### **[Reference](https://python.langchain.com/docs/how_to/routing/)**

In [ ]:
!pip install openai
!pip install langchain
!pip install langchain-openai

In [ ]:
import openai
import os

In [ ]:
f = open('/content/ts_openapi_key.txt')
api_key = f.read()
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key= os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
llm_model = "gpt-4o-mini" # This is a chat model
llm = ChatOpenAI(temperature=0, model=llm_model)

In [ ]:
prompt_classify = PromptTemplate.from_template("""Given the user question below, classify it as either being about
 `Physics`, `Math`, `Computer Science`.Do not respond with more than one word.
 <question> {question} </question> Classification:""")


In [ ]:
prompt_classify

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Given the user question below, classify it as either being about\n `Physics`, `Math`, `Computer Science`.Do not respond with more than one word.\n <question> {question} </question> Classification:')

In [ ]:
chain = prompt_classify  | llm | StrOutputParser()

In [ ]:
chain.invoke({"question":"What's a programming language?"})

'Computer Science'

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{question}
Start your answer with `Subject:Physics :`
"""
physics_prompt=ChatPromptTemplate.from_template(physics_template) # ChatPromptTemplate , PromptTemplate
physics_chain = physics_prompt | llm | StrOutputParser()

In [ ]:
math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{question}
Start your answer with `Subject:Math :`
"""
math_prompt=ChatPromptTemplate.from_template(math_template)
math_chain = math_prompt | llm | StrOutputParser()

In [ ]:
computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{question}
Start your answer with `Subject:Computer Science :`
"""

computer_science_prompt=ChatPromptTemplate.from_template(computerscience_template)
computer_science_chain = computer_science_prompt | llm | StrOutputParser()

In [ ]:
general_prompt=ChatPromptTemplate.from_template(
    """Respond to the following question:

Question: {question}
Start your answer with `Subject:General :`"""

)
general_chain = general_prompt | llm | StrOutputParser()

In [ ]:
def route(info):
    if "physics" in info["topic"].lower():
        return physics_chain
    elif "math" in info["topic"].lower():
        return math_chain
    elif "computer science" in info["topic"].lower():
        return computer_science_chain
    else:
        return general_chain

In [ ]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(route)
full_chain

{
  topic: PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Given the user question below, classify it as either being about\n `Physics`, `Math`, `Computer Science`.Do not respond with more than one word.\n <question> {question} </question> Classification:')
         | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7d89faffd390>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7d89faf8dd50>, root_client=<openai.OpenAI object at 0x7d89faffcbb0>, root_async_client=<openai.AsyncOpenAI object at 0x7d89faffd360>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))
         | StrOutputParser(),
  question: RunnableLambda(lambda x: x['question'])
}
| RunnableLambda(route)

In [ ]:
full_chain.invoke({"question":"What is black body radiation?"})

'Subject: Physics: \n\nBlack body radiation refers to the electromagnetic radiation emitted by an idealized object known as a "black body," which absorbs all incident radiation, regardless of frequency or angle. When a black body is in thermal equilibrium, it emits radiation in a characteristic spectrum that depends solely on its temperature, described by Planck\'s law. \n\nAs the temperature of the black body increases, the peak wavelength of the emitted radiation shifts to shorter wavelengths, a phenomenon described by Wien\'s displacement law. This concept is fundamental in understanding thermal radiation and has significant implications in various fields, including astrophysics and quantum mechanics.'

In [ ]:
full_chain.invoke({"question":"What's a programming language?"})

'Subject:Computer Science : \n\nA programming language is a formal set of instructions that can be used to produce various kinds of output, including software applications, algorithms, and data processing. It serves as a medium for humans to communicate with computers, allowing us to write code that the machine can interpret and execute.\n\nProgramming languages consist of syntax (rules for structure) and semantics (meaning of the statements). They can be categorized into high-level languages, which are more abstract and closer to human languages (like Python, Java, and C#), and low-level languages, which are closer to machine code (like Assembly and C).\n\nKey characteristics of programming languages include:\n\n1. **Syntax**: The set of rules that defines the combinations of symbols that are considered to be correctly structured programs.\n2. **Semantics**: The meaning of the syntactic elements and how they are interpreted by the machine.\n3. **Paradigms**: Different programming styl

In [ ]:
full_chain.invoke({"question":"What's 1+1"})

'Subject:Math :  \nTo solve the problem 1 + 1, we can break it down into its component parts:\n\n1. Identify the numbers involved: We have the number 1 and another number 1.\n2. Understand the operation: The operation we are performing is addition, which combines the values of the numbers.\n\nNow, we can add the two numbers together:\n1 + 1 = 2\n\nTherefore, the answer to the question is 2.'

In [ ]:
full_chain.invoke({"question":"What's a path integral?"}) #

'Subject: Physics : \n\nA path integral is a formulation of quantum mechanics that sums over all possible paths a particle can take between two points. Instead of considering a single trajectory, the path integral approach, developed by Richard Feynman, involves calculating the contribution of every conceivable path, each weighted by a factor related to the action (a quantity that encapsulates the dynamics of the system). \n\nMathematically, the path integral is expressed as an integral over all paths, and it provides a powerful way to compute quantum amplitudes. This method is particularly useful in quantum field theory and has applications in various areas of physics, including statistical mechanics and string theory. The concept emphasizes the probabilistic nature of quantum mechanics and highlights the idea that particles do not follow a single, well-defined path but rather explore all possible routes.'

In [ ]:
full_chain.invoke({"question":"tell me 2 facts about Taj Mahal India?"})

'Subject:General :  \n1. The Taj Mahal, located in Agra, India, was commissioned by Mughal Emperor Shah Jahan in memory of his beloved wife Mumtaz Mahal, who died during childbirth. Construction began in 1632 and was completed in 1653, making it a symbol of love and devotion.  \n\n2. The Taj Mahal is renowned for its stunning white marble architecture, which reflects different hues throughout the day, creating a mesmerizing visual effect. It is also a UNESCO World Heritage Site and attracts millions of visitors each year, making it one of the most famous landmarks in the world.'